In [78]:
# Native
import os
import re
import pprint

# Third Party
import censusdata
import pandas as pd

In [79]:
survey = "acs5"
year = 2018
state = "Missouri"
metro_area = "St. Louis"
MO_metro_counties = ["Franklin", "Jefferson", "Lincoln", "St. Charles", "St. Louis city", "St. Louis", "Warren"]
IL_metro_counties = ["Bond", "Calhoun", "Clinton", "Jersey", "Macoupin", "Madison", "Monroe", "St. Clair"]

In [80]:
try:
    census_api_key = os.environ["census_api_key"]
except:
    census_api_key = None

In [81]:
def fetch_metro_area_code(metro_area, survey=survey, year=year):
    geo = censusdata.censusgeo([('metropolitan statistical area/micropolitan statistical area', '*')])
    metro_areas = censusdata.geographies(geo, survey, year)
    for metro, code in metro_areas.items():
        if metro.startswith(metro_area):
            return re.search(r'\d+$', str(code)).group()
    return None

In [82]:
def fetch_state_code(state, survey=survey, year=year): 
    response = censusdata.geographies(censusdata.censusgeo([('state', '*')]), survey, year)
    state_values = str(response[state])
    state_code = re.search(r"\d+$", state_values)
    return state_code.group()

In [83]:
def fetch_county_code(county, state=state, survey=survey, year=year):
    response = censusdata.geographies(censusdata.censusgeo([('county', '*')]), survey, year)
    
    try:
        _county_ = f"{county} County, {state}"
        county_values = str(response[_county_])
    except Exception as err:
        _county_ = f"{county}, {state}"
        county_values = str(response[_county_])
        
    county_code = re.search(r"\d+$", county_values)
    return county_code.group()

In [84]:
def fetch_tract_codes_by_county(state_code, county_code, survey=survey, year=year):
    geo = censusdata.censusgeo([('state', state_code), ('county', county_code), ('tract', '*')])
    tracts = censusdata.geographies(geo, survey, year)
    tract_codes = []
    for tract, code in tracts.items():
        code = re.search(r'\d+$', str(code)).group()
        tract_codes.append((tract, code))
    return tract_codes

## Metro and State Codes

In [85]:
metro_code = fetch_metro_area_code(metro_area)
MO_state_code = fetch_state_code(state)
IL_state_code = fetch_state_code(state="Illinois")
print(f"STL Metro Code: {metro_code}")
print(f"MO State Code: {MO_state_code}")
print(f"IL State Code: {IL_state_code}")

STL Metro Code: 41180
MO State Code: 29
IL State Code: 17


## Missouri Counties

In [86]:
MO_counties = censusdata.geographies(censusdata.censusgeo([('state', MO_state_code), ('county', '*')]), survey, year)
pprint.pp(MO_counties)

{'Saline County, Missouri': censusgeo((('state', '29'), ('county', '195'))),
 'Madison County, Missouri': censusgeo((('state', '29'), ('county', '123'))),
 'Wright County, Missouri': censusgeo((('state', '29'), ('county', '229'))),
 'Vernon County, Missouri': censusgeo((('state', '29'), ('county', '217'))),
 'Nodaway County, Missouri': censusgeo((('state', '29'), ('county', '147'))),
 'Chariton County, Missouri': censusgeo((('state', '29'), ('county', '041'))),
 'Grundy County, Missouri': censusgeo((('state', '29'), ('county', '079'))),
 'Morgan County, Missouri': censusgeo((('state', '29'), ('county', '141'))),
 'Wayne County, Missouri': censusgeo((('state', '29'), ('county', '223'))),
 'Bollinger County, Missouri': censusgeo((('state', '29'), ('county', '017'))),
 'Cape Girardeau County, Missouri': censusgeo((('state', '29'), ('county', '031'))),
 'Shelby County, Missouri': censusgeo((('state', '29'), ('county', '205'))),
 'Webster County, Missouri': censusgeo((('state', '29'), ('cou

## Illinois Counties

In [ ]:
IL_counties = censusdata.geographies(censusdata.censusgeo([('state', IL_state_code), ('county', '*')]), survey, year)
pprint.pp(IL_counties)

## Metro County Codes (Missouri and Illinois)

In [88]:
MO_metro_county_codes = [fetch_county_code(county) for county in MO_metro_counties]
IL_metro_county_codes = [fetch_county_code(county, state="Illinois") for county in IL_metro_counties]
pprint.pp(dict(zip(MO_metro_counties, MO_metro_county_codes)))
pprint.pp(dict(zip(IL_metro_counties, IL_metro_county_codes)))

{'Franklin': '071',
 'Jefferson': '099',
 'Lincoln': '113',
 'St. Charles': '183',
 'St. Louis city': '510',
 'St. Louis': '189',
 'Warren': '219'}
{'Bond': '005',
 'Calhoun': '013',
 'Clinton': '027',
 'Jersey': '083',
 'Macoupin': '117',
 'Madison': '119',
 'Monroe': '133',
 'St. Clair': '163'}


## Metro Tract Codes (Saint Louis) LIMIT = 10

In [ ]:
MO_metro_tract_codes = [fetch_tract_codes_by_county(MO_state_code, county_code) for county_code in MO_metro_county_codes]
IL_metro_tract_codes = [fetch_tract_codes_by_county(IL_state_code, county_code) for county_code in IL_metro_county_codes]
pprint.pp(MO_metro_tract_codes[0])
pprint.pp(IL_metro_tract_codes)

In [ ]:
# censusdata.censustable('acs5', 2018, 'B24081')